In [ ]:
#관측 분포 → 전위 경사 → 구조 기울기 유도

import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import matplotlib.pyplot as plt

sim = Aer.get_backend('qasm_simulator')
shots = 1024
n = 3
steps = 30

# 초기 게이트 각도: 균형
U = np.full(n, np.pi / 4)
learning_rate = 0.1

# 기록
U_history = []

# 회로 실행
def run_circuit(U):
    qc = QuantumCircuit(n, n)
    for i in range(n):
        qc.ry(U[i], i)
    qc.barrier()
    qc.measure(range(n), range(n))
    result = execute(qc, sim, shots=shots).result()
    return result.get_counts()

# 분포 평균값 기반 업데이트
def update_U(U, counts):
    total = sum(counts.values())
    avg_bits = np.zeros(n)
    for bitstring, count in counts.items():
        bits = np.array([int(b) for b in bitstring[::-1]])  # Qiskit 비트 순서 보정
        avg_bits += bits * (count / total)
    delta = (avg_bits - 0.5) * np.pi * learning_rate
    return U + delta

for step in range(steps):
    counts = run_circuit(U)
    U_history.append(U.copy())
    U = update_U(U, counts)

# 시각화: 각 큐비트의 RY 각도 변화
U_history = np.array(U_history)

plt.figure(figsize=(10, 5))
for i in range(n):
    plt.plot(U_history[:, i], label=f'Qubit {i}')
plt.title("Gate Angle Adaptation via Distribution Gradient")
plt.xlabel("Step")
plt.ylabel("RY Angle (radians)")
plt.legend()
plt.grid(True)
plt.show()
